In [1]:
import pandas as pd
import numpy as np
import calendar
import warnings
from numba import jit 
import datetime
from time import time
from tqdm import tqdm_notebook as tqdm
from collections import Counter
from scipy import stats
from sklearn import preprocessing
import json
from sklearn.model_selection import GroupKFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, cohen_kappa_score
from typing import Any
import lightgbm as lgb
import xgboost as xgb
import json
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn import metrics
from itertools import product
import copy
import time
from functools import partial
import scipy as sp
from scipy import stats
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows",1000)
np.set_printoptions(precision=8)
warnings.filterwarnings("ignore")

In [2]:
def quadratic_weighted_kappa(y_pred, data):
    y_pred = np.transpose(y_pred.reshape(4,-1))
    y_pred = np.argmax(y_pred, axis=1)
    y_true = np.array(data.get_label()).astype("int")
    c_matrix = confusion_matrix(y_true, y_pred, labels=[0,1,2,3])

    numer = 0.0
    denom = 0.0
    
    for i in range(c_matrix.shape[0]):
        for j in range(c_matrix.shape[1]):
            n = c_matrix.shape[0]
            wij = ((i-j)**2.0)
            oij = c_matrix[i, j]
            eij = c_matrix[i, :].sum() * c_matrix[:, j].sum() / c_matrix.sum()
            numer += wij * oij
            denom += wij * eij
    return "Quadratic weighted Kappa", 1 - numer / denom, True

In [3]:
def qwk(a1, a2):
    max_rat = 3
    a1 = np.asarray(a1, dtype=int)
    a2 = np.asarray(a2, dtype=int)

    hist1 = np.zeros((max_rat + 1, ))
    hist2 = np.zeros((max_rat + 1, ))

    o = 0
    for k in range(a1.shape[0]):
        i, j = a1[k], a2[k]
        hist1[i] += 1
        hist2[j] += 1
        o +=  (i - j) * (i - j)

    e = 0
    for i in range(max_rat + 1):
        for j in range(max_rat + 1):
            e += hist1[i] * hist2[j] * (i - j) * (i - j)

    e = e / a1.shape[0]

    return np.round(1 - o / e, 8)

In [4]:
class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3])
        return -qwk(y, X_p)
    
    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        initial_coef = [0.5, 1.5, 2.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')
        
    def predict(self, X, coef):
        return pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3])

    def coefficients(self):
        return self.coef_['x']

# install

In [5]:
%%time
train = pd.read_csv('../input/data-science-bowl-2019/train.csv')
train_labels = pd.read_csv('../input/data-science-bowl-2019/train_labels.csv')
test = pd.read_csv('../input/data-science-bowl-2019/test.csv')
#specs = pd.read_csv('../input/data-science-bowl-2019/specs.csv')
sample_submission = pd.read_csv('../input/data-science-bowl-2019/sample_submission.csv')

CPU times: user 1min 8s, sys: 10.8 s, total: 1min 19s
Wall time: 1min 19s


In [6]:
keep_id = train[train.type == "Assessment"][['installation_id']].drop_duplicates()
train = pd.merge(train, keep_id, on="installation_id", how="inner")
train = train[train.installation_id.isin(train_labels.installation_id.unique())]

# Preprocess and Feature engineering

In [7]:
def encode_title(train, test, train_labels):
    # encode title
    train['title_event_code'] = list(map(lambda x, y: str(x) + '_' + str(y), train['title'], train['event_code']))
    test['title_event_code'] = list(map(lambda x, y: str(x) + '_' + str(y), test['title'], test['event_code']))
    all_title_event_code = list(set(train["title_event_code"].unique()).union(test["title_event_code"].unique()))
    # make a list with all the unique 'titles' from the train and test set
    list_of_user_activities = list(set(train['title'].unique()).union(set(test['title'].unique())))
    # make a list with all the unique 'event_code' from the train and test set
    list_of_event_code = list(set(train['event_code'].unique()).union(set(test['event_code'].unique())))
    list_of_event_id = list(set(train['event_id'].unique()).union(set(test['event_id'].unique())))
    # make a list with all the unique worlds from the train and test set
    list_of_worlds = list(set(train['world'].unique()).union(set(test['world'].unique())))
    # create a dictionary numerating the titles
    activities_map = dict(zip(list_of_user_activities, np.arange(len(list_of_user_activities))))
    activities_labels = dict(zip(np.arange(len(list_of_user_activities)), list_of_user_activities))
    activities_world = dict(zip(list_of_worlds, np.arange(len(list_of_worlds))))
    assess_titles = list(set(train[train['type'] == 'Assessment']['title'].value_counts().index).union(set(test[test['type'] == 'Assessment']['title'].value_counts().index)))
    # replace the text titles with the number titles from the dict
    train['title'] = train['title'].map(activities_map)
    test['title'] = test['title'].map(activities_map)
    train['world'] = train['world'].map(activities_world)
    test['world'] = test['world'].map(activities_world)
    train_labels['title'] = train_labels['title'].map(activities_map)
    win_code = dict(zip(activities_map.values(), (4100*np.ones(len(activities_map))).astype('int')))

    win_code[activities_map['Bird Measurer (Assessment)']] = 4110
    
    train['timestamp'] = pd.to_datetime(train['timestamp'])
    test['timestamp'] = pd.to_datetime(test['timestamp'])
    
    # identifier
    #train["identifier"] = train["event_data"].apply(lambda x: json.loads(x)["identifier"] if "identifier" in x else "None")
    #test["identifier"] = test["event_data"].apply(lambda x: json.loads(x)["identifier"] if "identifier" in x else "None")
    #list_of_identifier = list(set(train['identifier'].unique()).union(set(test['identifier'].unique())))
    #versions_identifier = dict(zip(list_of_identifier, np.arange(len(list_of_identifier))))
    #train["identifier"] = train["media_type"].map(versions_identifier)
    #test["identifier"] = test["media_type"].map(versions_identifier)
    
    return train, test, train_labels, win_code, list_of_user_activities, list_of_event_code, activities_labels, assess_titles, list_of_event_id, all_title_event_code

# get usefull dict with maping encode
train, test, train_labels, win_code, list_of_user_activities, list_of_event_code, activities_labels, assess_titles, list_of_event_id, all_title_event_code = encode_title(train, test, train_labels)
categoricals = ['session_title']

In [8]:
def get_data(user_sample, test_set=False):
    '''
    The user_sample is a DataFrame from train or test where the only one 
    installation_id is filtered
    And the test_set parameter is related with the labels processing, that is only requered
    if test_set=False
    '''
    # Constants and parameters declaration
    last_activity = 0
    user_activities_count = {'Clip':0, 'Activity': 0, 'Assessment': 0, 'Game':0}
    
    # news features: time spent in each activity
    time_spent_each_act = {actv: 0 for actv in list_of_user_activities}
    event_code_count = {eve: 0 for eve in list_of_event_code}
    last_session_time_sec = 0
    
    accuracy_groups = {0:0, 1:0, 2:0, 3:0}
    all_assessments = []
    accumulated_accuracy_group = 0
    accumulated_accuracy=0
    accumulated_correct_attempts = 0 
    accumulated_uncorrect_attempts = 0 
    accumulated_actions = 0
    accumulated_stumps = 0 
    accumulated_cataps = 0
    click_counts = 0
    accumulated_clicks = 0
    audio_counts = 0
    accumulated_audio_counts = 0
    animation_counts = 0
    accumulated_anime_counts = 0
    level = 0
    version = 0
    round_counts = 0
    accumulated_rounds = 0
    counter = 0
    time_first_activity = float(user_sample['timestamp'].values[0])
    durations = []
    
    # itarates through each session of one instalation_id
    for i, session in user_sample.groupby('game_session', sort=False):
        # i = game_session_id
        # session is a DataFrame that contain only one game_session
        
        # get some sessions information
        session_type = session['type'].iloc[0]
        session_title = session['title'].iloc[0]
        session_title_text = activities_labels[session_title] #from Andrew
        
        # get current session time in seconds
        if session_type != 'Assessment':
            time_spent = int(session['game_time'].iloc[-1] / 1000)
            time_spent_each_act[activities_labels[session_title]] += time_spent
            
            tmp_click_counts = session["event_data"].apply(lambda x: 1 if "\"coordinates\"" in x else 0)
            click_counts += np.sum(tmp_click_counts)
            
            tmp_media = session["event_data"].apply(lambda x: json.loads(x)["media_type"] if "\"media_type\"" in x else "None").value_counts()
            if "audio" in tmp_media.index:
                audio_counts += tmp_media["audio"]
            if "animation" in tmp_media.index:
                animation_counts += tmp_media["animation"]
                
            tmp_round = session["event_data"].apply(lambda x: json.loads(x)["rounds"] if "\"rounds\"" in x else 0)
            round_counts += np.sum(tmp_round)
                
            #level_list =  session["event_data"].apply(lambda x: json.loads(x)["level"] if "\"level\"" in x else 0)
            #tmp_level = np.max(level_list)
            #level = np.max([level, tmp_level])
            
            #version_list =  session["event_data"].apply(lambda x: json.loads(x)["version"] if "\"version\"" in x else 0)
            #tmp_version = np.max(version_list)
            #version = np.max([version, tmp_version])
            
        # for each assessment, and only this kind off session, the features below are processed
        # and a register are generated
        if (session_type == 'Assessment') & (test_set or len(session)>1):
            # search for event_code 4100, that represents the assessments trial
            # all_attempts doesn't include media_type, coordinate infomation in event_data
            all_attempts = session.query(f'event_code == {win_code[session_title]}')
            # then, check the numbers of wins and the number of losses
            true_attempts = all_attempts['event_data'].str.contains('true').sum()
            false_attempts = all_attempts['event_data'].str.contains('false').sum()
            
            tmp_stump_counts = all_attempts["event_data"].apply(lambda x: np.sum(json.loads(x)["stumps"]) if "\"stumps\"" in x else 0)
            stump_counts = np.sum(tmp_stump_counts)
            #tmp_catap_counts = all_attempts["event_data"].apply(lambda x: np.sum(json.loads(x)["caterpillars"]) if "\"caterpillars\"" in x else 0)
            #catap_counts = np.sum(tmp_catap_counts)

            # copy a dict to use as feature template, it's initialized with some itens: 
            # {'Clip':0, 'Activity': 0, 'Assessment': 0, 'Game':0}
            features = user_activities_count.copy()
            features.update(time_spent_each_act.copy())
            features.update(event_code_count.copy())
            # get installation_id for aggregated features
            features['installation_id'] = session['installation_id'].iloc[-1] #from Andrew
            # add title as feature, remembering that title represents the name of the game
            features['session_title'] = session['title'].iloc[0] 
            # the 4 lines below add the feature of the history of the trials of this player
            # this is based on the all time attempts so far, at the moment of this assessment
            features['accumulated_correct_attempts'] = accumulated_correct_attempts
            features['accumulated_uncorrect_attempts'] = accumulated_uncorrect_attempts
            accumulated_correct_attempts += true_attempts 
            accumulated_uncorrect_attempts += false_attempts
            
            features['accumulated_stumps'] = accumulated_stumps
            accumulated_stumps += stump_counts
            #features['accumulated_cataps'] = accumulated_cataps
            #accumulated_stumps += catap_counts
            accumulated_clicks += click_counts
            features['accumulated_clicks'] = accumulated_clicks
            click_counts = 0
            
            #features['highest_level'] = level
            #level = 0
            
            #features['highest_version'] = version
            #version = 0
            
            accumulated_audio_counts += audio_counts
            features['accumulated_audios'] = accumulated_audio_counts
            audio_counts = 0
            accumulated_anime_counts += animation_counts
            features['accumulated_animes'] = accumulated_anime_counts
            animation_counts = 0
            
            accumulated_rounds += round_counts
            features['accumulated_rounds'] = accumulated_rounds
            round_counts = 0
            
            # the time spent in the app so far
            if durations == []:
                features['duration_mean'] = 0
            else:
                features['duration_mean'] = np.mean(durations)
            durations.append((session.iloc[-1, 2] - session.iloc[0, 2]).seconds)
            # the accurace is the all time wins divided by the all time attempts
            features['accumulated_accuracy'] = accumulated_accuracy/counter if counter > 0 else 0
            accuracy = true_attempts/(true_attempts+false_attempts) if (true_attempts+false_attempts) != 0 else 0
            accumulated_accuracy += accuracy
            # a feature of the current accuracy categorized
            # it is a counter of how many times this player was in each accuracy group
            if accuracy == 0:
                features['accuracy_group'] = 0
            elif accuracy == 1:
                features['accuracy_group'] = 3
            elif accuracy == 0.5:
                features['accuracy_group'] = 2
            else:
                features['accuracy_group'] = 1
            features.update(accuracy_groups)
            accuracy_groups[features['accuracy_group']] += 1
            # mean of the all accuracy groups of this player
            features['accumulated_accuracy_group'] = accumulated_accuracy_group/counter if counter > 0 else 0
            accumulated_accuracy_group += features['accuracy_group']
            # how many actions the player has done so far, it is initialized as 0 and updated some lines below
            features['accumulated_actions'] = accumulated_actions
            
            # there are some conditions to allow this features to be inserted in the datasets
            # if it's a test set, all sessions belong to the final dataset
            # it it's a train, needs to be passed throught this clausule: session.query(f'event_code == {win_code[session_title]}')
            # that means, must exist an event_code 4100 or 4110
            if test_set:
                all_assessments.append(features)
            elif true_attempts+false_attempts > 0:
                all_assessments.append(features)
                
            counter += 1
        
        # this piece counts how many actions was made in each event_code so far
        n_of_event_codes = Counter(session['event_code'])
        
        for key in n_of_event_codes.keys():
            event_code_count[key] += n_of_event_codes[key]

        # counts how many actions the player has done so far, used in the feature of the same name
        accumulated_actions += len(session)
        if last_activity != session_type:
            user_activities_count[session_type] += 1
            last_activitiy = session_type
    # if test_set=True, only the last assessment must be predicted, the previous are scraped
    if test_set:
        return all_assessments[-1]
    # in train_set, all assessments are kept
    return all_assessments

In [9]:
new_train = []
for i, (ins_id, user_sample) in tqdm(enumerate(train.groupby('installation_id', sort=False)), total=train.installation_id.nunique(), desc='Installation_id', position=0):
    # user_sample is a DataFrame that contains only one installation_id
    new_train += get_data(user_sample)
new_train = pd.DataFrame(new_train)
print(new_train.shape)
del train


(17690, 108)


In [10]:
new_test = []
for ins_id, user_sample in tqdm(test.groupby('installation_id', sort=False), total=test.installation_id.nunique(), desc='Installation_id', position=0):
    a = get_data(user_sample, test_set=True)
    new_test.append(a)   
new_test = pd.DataFrame(new_test)
print(new_test.shape)
del test


(1000, 108)


In [11]:
X_train = new_train.drop(['accuracy_group'],axis=1) 
lbl = preprocessing.LabelEncoder()
lbl.fit(list(X_train["installation_id"]))
X_train["installation_id"] = lbl.transform(list(X_train["installation_id"]))
remove_features = [4235, 2070]
for i in X_train.columns:
    if X_train[i].std() == 0:
        remove_features.append(i)
X_train = X_train.drop(remove_features ,axis=1)
y_train = new_train.accuracy_group

# Modelling

In [12]:
# my method
n_folds=5
groups = np.array(X_train.installation_id)
gkf=GroupKFold(n_splits = n_folds)
coefficients = []
models = []
train_qwk_scores = []
test_qwk_scores = []
lgbm_params = {
    "objective" : "regression",
    "metric" : "rmse",
    "tree_learner": "serial",
    "max_depth" : 5,
    "boosting": 'gbdt',
    "num_leaves" : 13,
    "learning_rate" : 0.01,
}
#evals_result = {}
features_list = [i for i in X_train.columns if i != "installation_id"]
feature_importance_df = pd.DataFrame(features_list, columns=["Feature"])
for i , (train_index, test_index) in enumerate(gkf.split(X_train, y_train, groups)):
    optR = OptimizedRounder()
    X_train2 = X_train.iloc[train_index,:]
    y_train2 = y_train.iloc[train_index]
    X_test2 = X_train.iloc[test_index,:]
    y_test2 = y_train.iloc[test_index]
    X_train2 = X_train2.drop(['installation_id'],axis=1) 
    X_test2 = X_test2.drop(['installation_id'],axis=1) 
    lgb_train = lgb.Dataset(X_train2, y_train2)
    lgb_eval = lgb.Dataset(X_test2, y_test2, reference=lgb_train)
    
    clf = lgb.train(
        lgbm_params, lgb_train,
        valid_sets=lgb_eval,
        num_boost_round=100000,
        early_stopping_rounds=10,
        #evals_result=evals_result,
        #feval=quadratic_weighted_kappa,
    )
    
    models.append(clf)
    train_predict = clf.predict(X_train2, num_iteration = clf.best_iteration)
    test_predict = clf.predict(X_test2, num_iteration = clf.best_iteration)
    
    optR.fit(test_predict.reshape(-1,), y_test2)
    tmp_coefficients = optR.coefficients()
    print("fold_"+str(i)+" coefficients: ", tmp_coefficients)
    opt_train_preds = optR.predict(train_predict.reshape(-1, ), tmp_coefficients)
    train_qwk_score = qwk(y_train2, opt_train_preds)
    print("training qwk: ", train_qwk_score)
    opt_test_preds = optR.predict(test_predict.reshape(-1, ), tmp_coefficients)
    test_qwk_score = qwk(y_test2, opt_test_preds)
    print("validation qwk: ", test_qwk_score)
    train_qwk_scores.append(train_qwk_score)
    test_qwk_scores.append(test_qwk_score)
    coefficients.append(tmp_coefficients)
    
    feature_importance_df["Fold_"+str(i+1)] = clf.feature_importance()
    
feature_importance_df["Average"] = np.mean(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
feature_importance_df["Std"] = np.std(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
feature_importance_df["Cv"] = feature_importance_df["Std"] / feature_importance_df["Average"]

print("                             ")
print("-----------------------------")
print('coefficients: ', coefficients)
print('train qwk list:', train_qwk_scores)
print('train qwk average score:',np.mean(train_qwk_scores))
print('valid qwk list: ', test_qwk_scores)
print('valid qwk average score:',np.mean(test_qwk_scores))

[1]	valid_0's rmse: 1.25664
Training until validation scores don't improve for 10 rounds
[2]	valid_0's rmse: 1.25289
[3]	valid_0's rmse: 1.2492
[4]	valid_0's rmse: 1.24563
[5]	valid_0's rmse: 1.24206
[6]	valid_0's rmse: 1.23859
[7]	valid_0's rmse: 1.23514
[8]	valid_0's rmse: 1.23179
[9]	valid_0's rmse: 1.22841
[10]	valid_0's rmse: 1.22509
[11]	valid_0's rmse: 1.2219
[12]	valid_0's rmse: 1.2187
[13]	valid_0's rmse: 1.21562
[14]	valid_0's rmse: 1.21252
[15]	valid_0's rmse: 1.2095
[16]	valid_0's rmse: 1.20654
[17]	valid_0's rmse: 1.20363
[18]	valid_0's rmse: 1.20079
[19]	valid_0's rmse: 1.19796
[20]	valid_0's rmse: 1.19512
[21]	valid_0's rmse: 1.19239
[22]	valid_0's rmse: 1.18971
[23]	valid_0's rmse: 1.18708
[24]	valid_0's rmse: 1.18451
[25]	valid_0's rmse: 1.18199
[26]	valid_0's rmse: 1.17946
[27]	valid_0's rmse: 1.17699
[28]	valid_0's rmse: 1.17458
[29]	valid_0's rmse: 1.17228
[30]	valid_0's rmse: 1.16986
[31]	valid_0's rmse: 1.1677
[32]	valid_0's rmse: 1.16548
[33]	valid_0's rmse: 1.16

- train qwk list         : [0.58456667, 0.61070294, 0.58061442, 0.61492297, 0.6235943]
- train qwk average score: 0.60288026
- valid qwk list         :  [0.54205412, 0.53429504, 0.51877743, 0.53483738, 0.57686126]
- valid qwk average score: 0.5413650459999999
- 
- remove preprocess func
- train qwk list: [0.56328036, 0.59378594, 0.55295006, 0.59297017, 0.62745414]
- train qwk average score: 0.586088134
- valid qwk list:  [0.5271645, 0.53471058, 0.49851502, 0.51651786, 0.55895406]
- valid qwk average score: 0.527172404
- 
- add stump_sum 
- train qwk list: [0.57149591, 0.59305846, 0.56971176, 0.6252002, 0.61589187]
- train qwk average score: 0.59507164
- valid qwk list:  [0.5442096, 0.53769555, 0.51285398, 0.56447238, 0.55741725]
- valid qwk average score: 0.5433297519999999
- 
- add capapiler sum becomes worse
- 
- change CV and remove catap_sum
- train qwk list: [0.55487107, 0.5941021, 0.60636206, 0.59037453, 0.58405887]
- train qwk average score: 0.585953726
- valid qwk list:  [0.54117094, 0.53986498, 0.53609571, 0.5244176, 0.5083173]
- valid qwk average score: 0.529973306
- 
- add click
- train qwk list: [0.57970711, 0.5921434, 0.59581269, 0.58925578, 0.59387045]
- train qwk average score: 0.5901578860000001
- valid qwk list:  [0.55178162, 0.53365645, 0.52783459, 0.5330639, 0.50956325]
- valid qwk average score: 0.5311799620000001
- 
- add audio, animation counts
- train qwk list: [0.6108805, 0.59744852, 0.60218258, 0.58620582, 0.57717716]
- train qwk average score: 0.5947789159999999
- valid qwk list:  [0.58261239, 0.53025364, 0.54219853, 0.51770464, 0.50501624]
- valid qwk average score: 0.535557088
- 
- add highest level before assessment: becomes worse
- add total rounds before assessment: becomes worse
- remove 2070, but remain total rounds
- train qwk list: [0.56005611, 0.59647098, 0.62685213, 0.59336203, 0.62070229]
- train qwk average score: 0.5994887080000001
- valid qwk list:  [0.54688792, 0.53903335, 0.55462953, 0.53074222, 0.55291915]
- valid qwk average score: 0.5448424340000001

In [13]:
list(X_train.columns)

['Clip',
 'Activity',
 'Assessment',
 'Game',
 'Bug Measurer (Activity)',
 'All Star Sorting',
 'Air Show',
 'Bubble Bath',
 'Dino Dive',
 'Leaf Leader',
 'Bottle Filler (Activity)',
 'Egg Dropper (Activity)',
 'Sandcastle Builder (Activity)',
 'Happy Camel',
 'Fireworks (Activity)',
 'Pan Balance',
 'Scrub-A-Dub',
 'Crystals Rule',
 'Watering Hole (Activity)',
 'Flower Waterer (Activity)',
 'Dino Drink',
 'Chicken Balancer (Activity)',
 'Chow Time',
 2050,
 4100,
 4230,
 5000,
 2060,
 4110,
 5010,
 2075,
 2080,
 2081,
 2083,
 3110,
 4010,
 3120,
 3121,
 4020,
 4021,
 4022,
 4025,
 4030,
 4031,
 3010,
 4035,
 4040,
 3020,
 3021,
 4045,
 2000,
 4050,
 2010,
 2020,
 4070,
 2025,
 2030,
 4080,
 2035,
 2040,
 4090,
 4220,
 4095,
 'installation_id',
 'session_title',
 'accumulated_correct_attempts',
 'accumulated_uncorrect_attempts',
 'accumulated_stumps',
 'accumulated_clicks',
 'accumulated_audios',
 'accumulated_animes',
 'duration_mean',
 'accumulated_accuracy',
 0,
 1,
 2,
 3,
 'accumu

In [14]:
feature_importance_df.sort_values("Average", ascending=False).reset_index(drop=True).head(100)

,Feature,Fold_1,Fold_2,Fold_3,Fold_4,Fold_5,Average,Std,Cv
0,session_title,926,936,920,931,891,920.8,15.816447,0.017177
1,4070,568,467,472,621,453,516.2,66.384938,0.128603
2,accumulated_accuracy,360,404,311,480,350,381.0,57.674951,0.151378
3,Clip,405,371,400,364,346,377.2,22.265669,0.059029
4,3020,381,353,259,303,290,317.2,44.001818,0.138719
5,accumulated_accuracy_group,380,239,367,255,325,313.2,57.251725,0.182796
6,3121,190,276,199,297,291,250.6,46.400862,0.185159
7,2000,258,212,205,243,193,222.2,24.358982,0.109626
8,2010,176,198,162,231,182,189.8,23.616943,0.124431
9,4035,261,164,198,171,111,181.0,48.944867,0.270414


# prediction

In [15]:
# pattern 1: average regression outputs and predict class by avearge coefficients
#X_test = new_test.drop(["installation_id"], axis=1)
#X_test = X_test.drop(remove_features, axis=1)
#pred_value = np.zeros([X_test.shape[0]])
#test_coefficients = np.mean(coefficients, axis=0)
#for model in models:
#    pred_value += model.predict(X_test, num_iteration = model.best_iteration) /n_folds
#test_pred_class= optR.predict(pred_value.reshape(-1, ), tmp_coefficients)
#sample_submission["accuracy_group"] = test_pred_class
#sample_submission.to_csv('submission.csv', index=False)

In [16]:
# pattern 2: majority vote
X_test = new_test.drop(["installation_id"], axis=1)
X_test = X_test.drop(remove_features, axis=1)
pred_value = np.zeros([X_test.shape[0], 5])
for i, model in enumerate(models):
    tmp_pred_value=  model.predict(X_test, num_iteration = model.best_iteration)
    pred_value[:,i] = optR.predict(tmp_pred_value.reshape(-1, ), coefficients[i])
test_pred_class = pd.DataFrame(stats.mode(pred_value, axis=1)[0]).astype("int")
sample_submission["accuracy_group"] = test_pred_class
sample_submission.to_csv('submission.csv', index=False)